In [1]:
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import random
import json
import pandas as pd
import re


In [8]:
def get_keywordurl(keyword):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
        #上面是设置无界浏览器用的，不用打开浏览器也可以爬取数据！
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
        #对chrome开启开发者模式
    options.add_experimental_option('prefs', {'profile.managed_default_content_settings.images': 2})
    #不加载图片

    browser = webdriver.Chrome(options=options)
    wait =WebDriverWait(browser,50)#设置等待时间
    url = 'https://www.jd.com/'
    try:
        browser.get(url)
        input = wait.until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#key"))
                )  #等到搜索框加载出来
        submit = wait.until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#search > div > div.form > button"))
                )#等到搜索按钮可以被点击
        input.send_keys(keyword)#向搜索框内输入关键词
        submit.click()#点击

        wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "#J_bottomPage > span.p-skip > em:nth-child(1) > b")
            )) 
        keyword_url = browser.current_url
        browser.quit()
        return keyword_url 
    except:
        pass

def index_page(page,keyword_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
        #上面是设置无界浏览器用的，不用打开浏览器也可以爬取数据！
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
        #对chrome开启开发者模式
    options.add_experimental_option('prefs', {'profile.managed_default_content_settings.images': 2})
    #不加载图片

    browser = webdriver.Chrome(options=options)
    wait =WebDriverWait(browser,50)#设置等待时间
    
    browser.get(keyword_url)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(1)#设置延迟
    input=wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#J_bottomPage > span.p-skip > input"))
            )  #等到搜索框加载出来
    input.clear()
    input.send_keys(page)

    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#J_bottomPage > span.p-skip > a")))
    button.click()# 翻页动作

    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")#滑动到底部
    time.sleep(1)#设置随机延迟
    wait.until(
                EC.text_to_be_present_in_element((By.CSS_SELECTOR, "#J_bottomPage > span.p-num > a.curr"), str(page))
            )# 判断翻页成功,高亮的按钮数字与设置的页码一样
    wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#J_goodsList > ul"))
            )#等到60个商品都加载出来     

    html=browser.page_source
    browser.quit()
    return  html

def get_products(html):

    soup = BeautifulSoup(html, 'lxml')
    a=soup.find_all(class_='gl-i-wrap')
    dict={'title':[],'price':[],'shop':[],'commit':[]}
    for i in range(len(a)):
        #title
        if a[i].find('div',class_="p-name"):
              title=a[i].find('div',class_="p-name").find('em').text
        elif a[i].find('div',class_="p-name p-name-type-2"):
              title=a[i].find('div',class_="p-name p-name-type-2").find('em').text
        else:
              title=None
        dict['title'].append(title)

        #price
        pattern=re.compile('(\d+\.\d+)')
        if a[i].find('div',attrs={'class': 'p-price'}):
              price_text=a[i].find('div',attrs={'class': 'p-price'}).text
              price=re.findall(pattern,price_text)[-1]
         
        else:
              price==None
        dict['price'].append( price)

        #publication
        if a[i].find('span',class_="J_im_icon"):
              publication=a[i].find('span',class_="J_im_icon").text
        elif a[i].find('span',class_="curr-shop hd-shopname"):
              publication=a[i].find('span',class_="curr-shop hd-shopname").text
        elif a[i].find('a',class_="curr-shop hd-shopname"):
              publication=a[i].find('a',class_="curr-shop hd-shopname").text
        else:
              publication=None
        dict['shop'].append(publication)

        #commit
        if a[i].find('div',class_="p-commit"):
             commit=a[i].find('div',class_="p-commit").text.strip()[:-3]
        else:
             commit=None       
        dict['commit'].append(commit)
    return pd.DataFrame(dict)


def main(page_num):
    keyword_url=get_keywordurl("水果")
    Alltab=pd.DataFrame()
    for page in range(1,page_num+1):
        try:
            html=index_page(page=page,keyword_url=keyword_url)
            df=get_products(html)
            Alltab=pd.concat([Alltab,df],axis=0).reset_index(drop=True)
            print('第%d页信息爬取成功'%page)
        except:
            pass
    return Alltab
            

In [9]:
start=time.time()
Alltab=main(100)
end=time.time()
print('一共用了%d秒'%(end-start))
Alltab.to_csv('JDfruit.csv',encoding="utf-8_sig")

第1页信息爬取成功
第2页信息爬取成功
第3页信息爬取成功
第4页信息爬取成功
第5页信息爬取成功
第6页信息爬取成功
第7页信息爬取成功
第8页信息爬取成功
第9页信息爬取成功
第10页信息爬取成功
第11页信息爬取成功
第12页信息爬取成功
第13页信息爬取成功
第14页信息爬取成功
第15页信息爬取成功
第16页信息爬取成功
第17页信息爬取成功
第18页信息爬取成功
第19页信息爬取成功
第21页信息爬取成功
第22页信息爬取成功
第23页信息爬取成功
第24页信息爬取成功
第25页信息爬取成功
第26页信息爬取成功
第27页信息爬取成功
第28页信息爬取成功
第29页信息爬取成功
第30页信息爬取成功
第31页信息爬取成功
第32页信息爬取成功
第33页信息爬取成功
第34页信息爬取成功
第35页信息爬取成功
第36页信息爬取成功
第37页信息爬取成功
第38页信息爬取成功
第39页信息爬取成功
第40页信息爬取成功
第41页信息爬取成功
第42页信息爬取成功
第43页信息爬取成功
第44页信息爬取成功
第45页信息爬取成功
第46页信息爬取成功
第47页信息爬取成功
第48页信息爬取成功
第49页信息爬取成功
第50页信息爬取成功
第51页信息爬取成功
第52页信息爬取成功
第53页信息爬取成功
第54页信息爬取成功
第55页信息爬取成功
第56页信息爬取成功
第57页信息爬取成功
第58页信息爬取成功
第59页信息爬取成功
第60页信息爬取成功
第61页信息爬取成功
第62页信息爬取成功
第63页信息爬取成功
第64页信息爬取成功
第65页信息爬取成功
第66页信息爬取成功
第67页信息爬取成功
第68页信息爬取成功
第69页信息爬取成功
第70页信息爬取成功
第71页信息爬取成功
第72页信息爬取成功
第73页信息爬取成功
第74页信息爬取成功
第75页信息爬取成功
第76页信息爬取成功
第77页信息爬取成功
第78页信息爬取成功
第79页信息爬取成功
第80页信息爬取成功
第81页信息爬取成功
第82页信息爬取成功
第83页信息爬取成功
第84页信息爬取成功
第85页信息爬取成功
第86页信息爬取成功
第87页信息爬取成功
第88页信息爬取成功
第89页信息爬取成功
第90页信息爬取成功
第91页信息爬取成功
第92页信息爬取成功
第93页信息爬取

In [10]:
Alltab

,title,price,shop,commit
0,鼎端 陕西洛川苹果红富士 20个 单果约180~220g新鲜水果,78.00,顶端果业旗舰店,4.1万+
1,京东超市烟台红富士苹果 12个 净重2.6kg以上 单果190-240g 新鲜水果,54.90,水果京东自营专区,198万+
2,京东超市越南进口红心火龙果 4个装 单果约330-420g 新鲜水果,46.90,水果京东自营专区,184万+
3,京觅 精选特级三红蜜柚 红心柚子4粒装 净重约5-6kg 新鲜水果礼盒装,59.90,水果京东自营专区,193万+
4,京东超市金秋红蜜 水蜜桃 新鲜桃子 精选特级果3kg装 铂金果 新鲜水果,59.90,水果京东自营专区,70万+
...,...,...,...,...
5935,玖原农珍 广西百香果 3kg水果 大果60-90g,36.80,玖原农珍旗舰店,2400+
5936,嘉食和【精品大果】徐香猕猴桃24枚单果130-150克新鲜生鲜水果陕西眉县绿心奇异果 果园现摘现发,58.50,嘉食和旗舰店,3800+
5937,东北花盖梨2.5kg 新鲜水果特产 软梨梨子冻梨 5斤装,39.00,金腰芒官方旗舰店,100+
5938,【大果】沙糖桔东江湖蜜桔5斤果径50mm以上新鲜橘子无籽蜜橘当季水果冰糖橘新鲜超甜桔子无核柑...,22.90,务农妹旗舰店,6


In [1]:
import numpy as np
import pandas as pd 

In [44]:
a=pd.read_excel('论文回归数据.xlsx')

In [45]:
a.head()

,城市,location,city,时间,住宅价格（元/平方米）,AQI,住宅投资（亿),人均gdp(元）,第三产业比值（%）,城市每万人拥有在校大学生人,年末总人口（万人）,竣工,销售
0,北京市,1,1,2013,9.76,4.61,7.42,11.83,4.37,6.12,7.18,7.433690,7.217935
1,北京市,1,1,2014,9.78,4.83,7.47,11.84,4.36,6.12,7.20,7.497950,7.035735
2,北京市,1,1,2015,9.95,4.80,7.48,11.84,4.37,6.11,7.20,7.228548,7.027173
3,北京市,1,1,2016,10.18,4.73,7.48,11.87,4.37,6.09,7.22,7.144455,6.888950
4,北京市,1,1,2017,10.34,4.63,7.34,11.89,4.37,6.08,7.21,6.403640,6.411457


In [46]:
a['id']=a.index

In [49]:
a['city']=a['城市']
a['year']=a['时间']
a['price']=a['住宅价格（元/平方米）']

In [55]:
b=a[['id','city','year','price']]

In [57]:
b.to_csv('paper.csv',index=None)

In [63]:
a.drop(['城市','时间'],axis=1)

,location,city,住宅价格（元/平方米）,AQI,住宅投资（亿),人均gdp(元）,第三产业比值（%）,城市每万人拥有在校大学生人,年末总人口（万人）,竣工,销售,id,year,price
0,1,北京市,9.76,4.61,7.42,11.83,4.37,6.12,7.18,7.433690,7.217935,0,2013,9.76
1,1,北京市,9.78,4.83,7.47,11.84,4.36,6.12,7.20,7.497950,7.035735,1,2014,9.78
2,1,北京市,9.95,4.80,7.48,11.84,4.37,6.11,7.20,7.228548,7.027173,2,2015,9.95
3,1,北京市,10.18,4.73,7.48,11.87,4.37,6.09,7.22,7.144455,6.888950,3,2016,10.18
4,1,北京市,10.34,4.63,7.34,11.89,4.37,6.08,7.21,6.403640,6.411457,4,2017,10.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0,重庆市,8.54,4.87,7.60,10.42,3.85,5.28,8.12,7.961178,8.380042,170,2013,8.54
171,0,重庆市,8.49,4.53,7.76,10.43,3.85,5.32,8.12,7.927162,8.394727,171,2014,8.49
172,0,重庆市,8.46,4.39,7.72,10.43,3.87,5.36,8.12,8.066490,8.406867,172,2015,8.46
173,0,重庆市,8.47,4.39,7.68,10.44,3.87,5.38,8.13,8.033983,8.538066,173,2016,8.47
